In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from tqdm.notebook import tqdm, trange
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
from google.colab import drive
import os
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
zip_file_path = '/content/drive/MyDrive/ROP_datasets/annotations.zip'
# Распаковать архив в текущую папку
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall()

In [54]:
class ROP_Dataset(Dataset):
    def __init__(self, txt_file, image_dir, transform=None):
        self.image_list = pd.read_csv(txt_file, header=None)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        img_tag = self.image_list.iloc[idx, 1]
        img_tag =  np.array([img_tag], dtype=float).reshape(-1)
        img_path = os.path.join(self.image_dir, self.image_list.iloc[idx, 0])
        image = np.array(Image.open(img_path))
        sample = {"image": image, "tag": img_tag}

        if self.transform is not None:
            image = self.transform(image)


        return sample

In [55]:
# Create Dataset Training
dataset_train = ROP_Dataset(txt_file='./annotations/train.txt', image_dir='./annotations/images')
# Create Dataset Testing
dataset_test = ROP_Dataset(txt_file='./annotations/test.txt', image_dir='./annotations/images')
# Create Dataset Validation
dataset_valid = ROP_Dataset(txt_file='./annotations/valid.txt', image_dir='./annotations/images')

# DataLoader Training
dataloader_train = DataLoader(dataset_train, batch_size=32, shuffle=True)
# DataLoader Testing
dataloader_test = DataLoader(dataset_test, batch_size=32, shuffle=True)
# DataLoader Validation
dataloader_valid = DataLoader(dataset_valid, batch_size=32, shuffle=True)

print(next(iter(dataloader_train)))
print(next(iter(dataloader_test)))
print(dataloader_valid)

{'image': tensor([[[[41, 25, 28],
          [40, 24, 27],
          [40, 26, 26],
          ...,
          [39, 25, 22],
          [38, 27, 23],
          [37, 26, 20]],

         [[39, 24, 29],
          [37, 22, 25],
          [37, 25, 27],
          ...,
          [37, 26, 24],
          [38, 27, 23],
          [36, 27, 20]],

         [[39, 28, 34],
          [37, 26, 30],
          [41, 30, 34],
          ...,
          [33, 23, 24],
          [36, 26, 24],
          [35, 28, 22]],

         ...,

         [[34, 31, 26],
          [32, 28, 25],
          [32, 27, 31],
          ...,
          [33, 30, 21],
          [33, 25, 22],
          [32, 24, 21]],

         [[32, 28, 27],
          [35, 31, 30],
          [33, 29, 28],
          ...,
          [33, 29, 26],
          [31, 26, 23],
          [30, 25, 22]],

         [[32, 28, 29],
          [34, 30, 29],
          [31, 30, 28],
          ...,
          [29, 28, 26],
          [31, 25, 25],
          [30, 24, 24]]],


       

In [57]:
for i, sample in enumerate(dataset_train):
    print(i, sample['image'].shape, sample['tag'].shape)
    if i == 3: break

0 (1200, 1600, 3) (1,)
1 (1200, 1600, 3) (1,)
2 (1200, 1600, 3) (1,)
3 (1200, 1600, 3) (1,)


In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')

Using cuda for inference


In [ ]:
efficientnet = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b0', pretrained=True)
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_convnets_processing_utils')

efficientnet.eval().to(device)